# Imports, Vars, Functions

In [1]:
import copy
import random
from datetime import datetime, timedelta
from importlib.resources import files
from pprint import pprint, pformat
from time import time

from ledhntr import LEDHNTR
from ledhntr.data_classes import(
    Attribute, Entity, Relation, Thing, Query
)

led = LEDHNTR()
tdb = led.plugins['typedb_client']
SCHEMA = str(files('ledhntr').joinpath('schemas/schema.tql'))
nukeit=True

# ; Generators

def gen_ips(num, existing_ips=[]):
    for i in range(num):
        ip = ".".join(str(random.randint(0,255)) for _ in range(4))
        if ip not in existing_ips:
            existing_ips.append(ip)
    if len(existing_ips) >= num:
        return existing_ips
    missing = num-len(existing_ips)
    if missing:
        print(f"Missing {missing} IPs - generating more...")
    existing_ips = gen_ips(missing, existing_ips)
    return existing_ips

def gen_lorem(words, num_sentences=1):
    sentences = []
    for s in range(num_sentences):
        lorem_ipsum_text = (
            "Lorem ipsum dolor sit amet consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore "
            "magna aliqua Ut enim ad minim veniam quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo "
            "consequat Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur "
            "Excepteur sint occaecat cupidatat non proident sunt in culpa qui officia deserunt mollit anim id est laborum"
        )
        words_list = lorem_ipsum_text.split()
        output_text = ' '.join(random.choice(words_list) for _ in range(words))
        output_text += "."
        sentences.append(f"{output_text}")
    return ' '.join(sentences)

def roll_die(sides=6):
    return random.randint(1,sides)

def gen_hunts(num_hunts=1, num_ips=50):
    all_hunts = []
    for nh in range(num_hunts):
        ips = gen_ips(num_ips)
        now = datetime.now()
        yesterday = now - timedelta(days=1)
        tomorrow = now + timedelta(days=1)

        '''
        date_discovered = Attribute(label='date-discovered', value=yesterday)
        date_seen_yest = Attribute(label='date-seen', value=yesterday)
        date_seen_today = Attribute(label='date-seen', value=now)
        date_seen_tom = Attribute(label='date-seen', value=tomorrow)
        '''
        asn = Attribute(label='note', value='TRASH-PANDA-SERVERS')

        ipents = []

        for ip in ips:
            ipattr = Attribute(label='ip-address', value=ip)
            # ipent = Entity(label='ip', has=[ipattr, date_discovered, date_seen_yest, date_seen_today, date_seen_tom])
            ipent = Entity(label='ip', has=[ipattr])
            roll = roll_die(3)
            if roll==3:
                ipent.has.append(asn)
            else:
                randasn = gen_lorem(3).replace(' ', '-').upper()
                randasnattr = Attribute(label='note', value=randasn)
                ipent.has.append(randasnattr)
            ipents.append(ipent)

        nowint = int(time())
        hunt_name = Attribute(label='hunt-name', value=f'HUNT-{nh}-{nowint}')
        hunt_active = Attribute(label='hunt-active', value=True)
        hunt = Relation(label='hunt', has=[hunt_name, hunt_active], players={'related': ipents})
        # pprint(hunt)
        # pprint(hunt.players['related'][0].to_dict())
        all_hunts.append(hunt)
    return all_hunts

def gen_timestamps(count, start_year=2015, end_year=2023):
    start_date = datetime(start_year, 1, 1)
    end_date = datetime(end_year, 12, 31)
    time_diff = end_date - start_date

    # return [(start_date + timedelta(days=random.randint(0, time_diff.days), 
    #                                 seconds=random.randint(0, 86399))).strftime('%Y-%m-%d %H:%M:%S') 
    return [(start_date + timedelta(days=random.randint(0, time_diff.days), 
                                    seconds=random.randint(0, 86399)))
            for _ in range(count)]

def ts_to_ents(timestamps):
    time_ents = []
    for ts in timestamps:
        year = Attribute(label='year', value=ts.year)
        month = Attribute(label='month', value=ts.month)
        day = Attribute(label='day', value=ts.day)
        hour = Attribute(label='hour', value=ts.hour)
        minute = Attribute(label='minute', value=ts.minute)
        second = Attribute(label='second', value=ts.second)
        time_ent = Entity(label='utc', has=[year,month,day,hour,minute,second])
        time_ents.append(time_ent)
    return time_ents

def ts_to_attrs(timestamps):
    attrs = []
    for ts in timestamps:
        attr = Attribute(label='date-seen', value=ts)
        attrs.append(attr)
    return attrs

2024-05-07 11:54:06 [INFO] ledhntr[26560] > core.py > (_reload_all_plugins) [158] >  Loading auto_hunter...
2024-05-07 11:54:06 [INFO] ledhntr[26560] > core.py > (_reload_all_plugins) [167] >  Successfully loaded auto_hunter!
2024-05-07 11:54:06 [INFO] ledhntr[26560] > core.py > (_reload_all_plugins) [158] >  Loading censys...
2024-05-07 11:54:06 [INFO] ledhntr[26560] > core.py > (_reload_all_plugins) [167] >  Successfully loaded censys!
2024-05-07 11:54:06 [INFO] ledhntr[26560] > core.py > (_reload_all_plugins) [158] >  Loading compare_things...
2024-05-07 11:54:06 [INFO] ledhntr[26560] > core.py > (_reload_all_plugins) [167] >  Successfully loaded compare_things!
2024-05-07 11:54:06 [INFO] ledhntr[26560] > core.py > (_reload_all_plugins) [158] >  Loading hyas...
2024-05-07 11:54:06 [INFO] ledhntr[26560] > core.py > (_reload_all_plugins) [167] >  Successfully loaded hyas!
2024-05-07 11:54:06 [INFO] ledhntr[26560] > core.py > (_reload_all_plugins) [158] >  Loading jsonflats_client...
2

# Generate Databases
One for datetime attributes like normal, one for "fast-indexing" described here: https://medium.com/vaticle/modelling-time-within-a-strongly-typed-database-55ba91ecad62

In [2]:
# DateTime Attributes (current method for TypeDB)
DB_NAME = "DATETIME-ATTRIBUTES"
tdb.db_name=DB_NAME
if nukeit:
    tdb.delete_db(DB_NAME)
if not tdb.check_db(DB_NAME):
    tdb.create_db()
    tdb.write_tql_file(file=SCHEMA)

# FAST-INDEX method
DB_NAME = "FAST-INDEX-TIMES"
tdb.db_name=DB_NAME
if nukeit:
    tdb.delete_db(DB_NAME)
if not tdb.check_db(DB_NAME):
    tdb.create_db()
    tdb.write_tql_file(file=SCHEMA)

# Create hunt
hunts = gen_hunts(1, 1)
attr_hunt = copy.deepcopy(hunts[0])
ents_hunt = copy.deepcopy(hunts[0])

# Generate timestamps
timestamps = gen_timestamps(250, start_year=2021, end_year=2023)

# Generate DateTime Attributes
dtattrs = ts_to_attrs(timestamps)

# Generate FastIndex UTC Entities
dtents = ts_to_ents(timestamps)

# Attach attributes to hunt
attr_hunt.has += dtattrs

# Attach fast-index time entities to hunt
ents_hunt.players['related']+=dtents

# Write hunt to attr database
tdb.db_name = "DATETIME-ATTRIBUTES"
tdb.add_relation(attr_hunt)

# Write hunt to fast database
tdb.db_name = "FAST-INDEX-TIMES"
tdb.add_relation(ents_hunt)

2024-05-07 11:54:40 [INFO] ledhntr[26560] > typedb_client.py > (delete_db) [1728] >  Deleting DATETIME-ATTRIBUTES...
2024-05-07 11:54:40 [INFO] ledhntr[26560] > typedb_client.py > (write_tql_file) [3713] >  Writing TQL file C:\Users\drive\OneDrive\Documents\GitHub\ledhntr-suite-public\ledhntr\ledhntr\schemas\schema.tql...
2024-05-07 11:54:41 [INFO] ledhntr[26560] > typedb_client.py > (delete_db) [1728] >  Deleting FAST-INDEX-TIMES...
2024-05-07 11:54:41 [INFO] ledhntr[26560] > typedb_client.py > (write_tql_file) [3713] >  Writing TQL file C:\Users\drive\OneDrive\Documents\GitHub\ledhntr-suite-public\ledhntr\ledhntr\schemas\schema.tql...
2024-05-07 11:54:42 [INFO] ledhntr[26560] > typedb_client.py > (add_entity) [330] >  Adding Entity <Entity(label=ip,con=0.0,ip-address=206.127.173.66,has=5)
2024-05-07 11:54:42 [INFO] ledhntr[26560] > typedb_client.py > (add_relation) [400] >  Adding Relation <Relation(label=hunt,hunt-name=HUNT-0-1715097281,has=253,active_roles=1,total_players=1)
2024-0

True

# Test Query Times

In [3]:
so = Relation(label='hunt')

def clock_search(search_object=None, client=None):
    stime = time()
    led.logger.info(f"#### {tdb.db_name} ####")
    led.logger.info(f"START SEARCH {stime}")
    res = tdb.find_things(search_object)
    etime = time()
    led.logger.info(f"END SEARCH {etime}")
    led.logger.info(f"### {tdb.db_name} RESULTS ###")
    led.logger.info(f"### {etime-stime} SECONDS ###")
    return res

tdb.db_name = "DATETIME-ATTRIBUTES"
attr_res = clock_search(so, tdb)

tdb.db_name = "FAST-INDEX-TIMES"
ent_res = clock_search(so, tdb)

tdb.db_name = "DATETIME-ATTRIBUTES"
attr_res = clock_search(so, tdb)

tdb.db_name = "FAST-INDEX-TIMES"
ent_res = clock_search(so, tdb)

2024-05-07 11:56:26 [INFO] ledhntr[26560] > 2108037218.py > (clock_search) [5] >  #### DATETIME-ATTRIBUTES ####
2024-05-07 11:56:26 [INFO] ledhntr[26560] > 2108037218.py > (clock_search) [6] >  START SEARCH 1715097386.0713677
2024-05-07 11:56:26 [INFO] ledhntr[26560] > typedb_client.py > (check_tx) [1260] >  Session mismatch. Looking for DATETIME-ATTRIBUTES got FAST-INDEX-TIMES! Creating new session.
2024-05-07 11:56:26 [INFO] ledhntr[26560] > 2108037218.py > (clock_search) [9] >  END SEARCH 1715097386.2557852
2024-05-07 11:56:26 [INFO] ledhntr[26560] > 2108037218.py > (clock_search) [10] >  ### DATETIME-ATTRIBUTES RESULTS ###
2024-05-07 11:56:26 [INFO] ledhntr[26560] > 2108037218.py > (clock_search) [11] >  ### 0.1844174861907959 SECONDS ###
2024-05-07 11:56:26 [INFO] ledhntr[26560] > 2108037218.py > (clock_search) [5] >  #### FAST-INDEX-TIMES ####
2024-05-07 11:56:26 [INFO] ledhntr[26560] > 2108037218.py > (clock_search) [6] >  START SEARCH 1715097386.2593694
2024-05-07 11:56:26 [INF

In [4]:
print(f"Attribute results")
pprint(attr_res)
print(f"Entity Results")
pprint(ent_res)

Attribute results
[<Relation(label=hunt,iid=0x847080088000000000000000,hunt-name=HUNT-0-1715097281,has=253,active_roles=1,total_players=1)]
Entity Results
[<Relation(label=hunt,iid=0x847080088000000000000000,hunt-name=HUNT-0-1715097281,has=3,active_roles=1,total_players=251)]


In [7]:
ent_res[0].players['related'][4].has

[<Attribute(label=hour,value=15.0),
 <Attribute(label=ledid,value=utc_1715097281344_ce1e0f),
 <Attribute(label=minute,value=49.0),
 <Attribute(label=day,value=6.0),
 <Attribute(label=month,value=11.0),
 <Attribute(label=second,value=40.0),
 <Attribute(label=year,value=2023.0),
 <Attribute(label=date-discovered,value=2024-05-07 19:54:42+00:00),
 <Attribute(label=confidence)]